# DASK to ServiceX - Using a Task Chain

In this demo we'll take advantage of DASK and ServiceX. This work is driven by the fact that `AwkwardInputLayer` seems like it will not take tasks as inputs. So we need to move onto something else.

## Assumptions:

* We don't start anything until we know the number of files that SX will produce. Thus we know the number of partitions up front.
* We are ok with some files failing coming out of SX
* We are going to do one partition per file
* When we start we don't know all the _names_ of the files produced.
* We have to fetch them from minio, and that doesn't put them in any sort of time order.

## Design Outline

* A single `dask` task/layer that has a single output per partition. The output is just a string, indicating the file we need to open in uproot.
* A single thread polls ServiceX looking for new files to show up, and when they do, it passes them to a `dask` output. A `dask` task then tries to open the file.
* We can't use a queue for the downstream tasks to grab - so we'll use a task chain.

The problem:
* `dask` is built around the idea that any task can be re-run with the exact same results.
* In this implementation we are using a chain of tasks to emulate a distributed queue.
* But the basic problem ends up being: the thread that inserts the items on the `queue` might be in a different process than the `task` that needs to read the `queue`.
* `dask` points this out by refusing to pickle a lock.

## Imports

In [1]:
import dask_awkward as dak
import awkward as ak
import dask
import uproot

from dask.highlevelgraph import Layer, HighLevelGraph
from dask.distributed import Client, LocalCluster
from typing import AbstractSet

import logging

import threading
import queue
import time

# Make debugging a little easier...
cluster = LocalCluster(processes=False)
client = Client(cluster)

c:\Users\gordo\Code\iris-hep\awkward-20-testing\.venv\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64850 instead
  warnings.warn(


## The `uproot.dask` getting the form info

Eventually this gets built from scratch and the query. But lets not worry about that now.

### The `awkward.Form` file form

We need the form from the schema to prevent us from having to open files that do not yet exist in hour hack. Eventually we'll have to build this from the schema we know exists from the `func_adl` query.

In [2]:
dummy_filename = "0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1"
with uproot.open(dummy_filename) as file:
    file_form = file['treeme'].arrays().layout.form
    metadata = dak.core.typetracer_array(file['treeme'].arrays())

file_form, metadata

(RecordForm([ListOffsetForm('i64', NumpyForm('float64'))], ['JetPt']),
 <Array-typetracer [...] type='## * {JetPt: var * float64}'>)

Next, lets test it.

In [3]:
test_ar = uproot.dask({dummy_filename: "treeme"}, open_files=False, known_base_form=file_form)
test_ar.JetPt.compute()

<Array [[102, 63.1], [...], ..., [153, 90.5, 54]] type='50000 * var * float64'>

## The Task Chain

Create a layer that emulates pulling data off SX.

We will emulate the return from SX via the `sd_filelist` - this will contains all the files that we'd ask SX for return from a query. We'll use dummy files to keep ourselves "sane" but will always access `dummy_filename` in the end b.c. that is the actual file we have locally in this repo!

In [4]:
dummy_filename = "0fc6e51a5ea6dea107c195591d20a1b2-15.26710677._000019.pool.root.1"
sd_filelist = [
    f"file_{i}" for i in range(5)
]

In [5]:
import threading
import multiprocessing
import time
from typing import List, Optional, Tuple

class SXFileFetcher(Layer):
    '''Grab new files from SX and if they haven't been assigned, assign one to our output. Also pass on
    the list of all assigned files so the next step knows what files haven't been assigned yet.

    * If we are the first, then we do not need to wait for any dependencies
    '''
    def __init__(self, name: str, previous_output: Optional[str] = None):
        '''Wait for at least one new file to become available from SX. Send it out in the output.

        Args:
            name (str): Name of the basics for the fetch. The actual layer name will be this plus the file number.
            previous_output (str): The name of the previous output. If None, then we are the first and do not need to wait for any dependencies.
        '''
        super().__init__()
        self.name = name

        self.tasks = {}
        if previous_output is None:
            self.tasks[self.name] = (self.fetch_file,)
        else:
            self.tasks = {name: (self.fetch_file, previous_output)}

    def __getitem__(self, key):
        return self.tasks[key]

    def __iter__(self):
        return iter(self.tasks)

    def __len__(self):
        return len(self.tasks)

    def is_materialized(self):
        return False
    
    def get_output_keys(self) -> AbstractSet[str | bytes | int | float]:
        return set(self.tasks.keys())
    
    def fetch_file(self, previous_result: Tuple[str, List[str]] = ("", [])) -> Tuple[str, List[str]]:
        '''Fetch a file from SX and return it along with the list of all files that have been assigned so far.

        Args:
            previous_result (Tuple[str, List[str]]): The previous result. If None, then we are the first.

        Returns:
            Tuple[str, List[str]]: The file that was fetched and the list of all files that have been assigned so far.
        '''
        # Get the result from SX and then look for a new file.
        _, all_started_files = previous_result
        started_files = set(all_started_files)

        sx_files = set(sd_filelist)
        new_files = sx_files - started_files

        assert len(new_files) > 0, "No new files, but should be - in reality we would now poll SX"
        new_file = new_files.pop()

        # Now, build the info we pass on down.
        return (new_file, all_started_files + [new_file])

And the layer that will load files from the above.

In [6]:
class URLoaderLayer(Layer):
    def __init__(self, name, sx_layer_name, output_names):
        super().__init__()
        self.name = name
        # self.dependencies = {name: sx_layer_name}
        self.tasks = {
            (name, i): (lambda file_name: self.get_data(file_name), f'{f_name}')
            for i, f_name in enumerate(output_names)
        }

    def __getitem__(self, key):
        return self.tasks[key]

    def __iter__(self):
        return iter(self.tasks)

    def __len__(self):
        return len(self.tasks)

    def is_materialized(self):
        return False
    
    def get_output_keys(self) -> AbstractSet[str | bytes | int | float]:
        return set(self.tasks.keys())
    
    def get_data(self, file_info: Tuple[str, List[str]]) -> ak.Array:
        '''Return the info that is needed by uproot to actually open the file'''
        # This message swallowed unless we use a dask `LocalCluster` (use it for debugging)
        logging.warning(f"Returning info for file {file_info[0]}")
        with uproot.open(dummy_filename) as file:
            return file["treeme"].arrays()

We know ahead of time how many "files" there are in a query - so we can use that to build up all those layers that do the task chain.

In [11]:
# The layers that will each pick off a file.
n_files_in_sx_query = len(sd_filelist)
file_finder_layers = [
    SXFileFetcher(f"sx_fetcher_{i}", f"sx_fetcher_{i-1}" if i > 0 else None)
    for i in range(n_files_in_sx_query)
]
output_names = [s for ss in file_finder_layers for s in ss.get_output_keys()]
assert len(output_names) == n_files_in_sx_query, "We should have one output per file finder layer"

# The layer that does the loading has to be hooked up to all these previous layers.
loader_layer = URLoaderLayer("uproot_loader", "ur-loader-layer", output_names)

# Now, the high level layer...
hlg = HighLevelGraph(
    layers={f.name: f for f in file_finder_layers + [loader_layer]},
    dependencies={
        **{loader_layer.name: {s.name for s in file_finder_layers}},
        **{s.name: set() for s in file_finder_layers}
    }
)

# And finally the array...
ar = dak.core.new_array_object(hlg, "uproot_loader", meta=metadata, npartitions=n_files_in_sx_query)

In [12]:
ar.JetPt[ar.JetPt > 100.0].compute()

<Array [[102], [], [110], ..., [114], [153]] type='250000 * var * float64'>

## Downsides

This will work. Some issues...

* The SX query will be made from anywhere in the cluster. So the user's `servicex.yaml` file will need to be available everywhere. Or it would have to be made available in every client.
* With 1000's of files, you'll get tuples that are 1000 URL's long being passed around inside of DASK (about 150Kb max, I would guess).

But some really nice things:

* The data load from `minio` will be from across the cluster - no single machine will be doing this - so a very distributed way to load the data.